In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cd /content/drive/MyDrive/332_5-2_HD

In [ ]:
!pip install git+https://github.com/RVC-Project/Retrieval-based-Voice-Conversion

In [ ]:
import pandas as pd
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt

from rvc.modules.vc.modules import VC
from dotenv import load_dotenv
from scipy.io import wavfile
from pathlib import Path

In [ ]:
load_dotenv("/content/drive/MyDrive/332_5-2_HD/.env", override=True)

In [ ]:
def rvc_audio(model_path, input_speech_path, output_speech_path):
  load_dotenv("/content/drive/MyDrive/332_5-2_HD/.env", override=True)
  vc = VC()
  vc.get_vc(Path(model_path))
  tgt_sr, audio_opt, times, _ = vc.vc_inference(1, input_speech_path)
  wavfile.write(output_speech_path, tgt_sr, audio_opt)

In [ ]:
rvc_audio("/content/drive/MyDrive/332_5-2_HD/models/lumine/lumine.pth",
          "/content/drive/MyDrive/332_5-2_HD/real/visal.wav",
          "/content/drive/MyDrive/332_5-2_HD/fake/visal-lumine.wav"
          )

In [ ]:
def get_features_(input_audio_path):
    #y, sr = librosa.load(input_audio_path, sr=None)
    y = AudioSegment.from_wav(input_audio_path)
    x = y.get_array_of_samples() 
    sr = y.frame_rate
    print("sr: ", sr)
    block_length = sr # 48000 samples = 1 second
    overlap_constant = 0.6354
    hop_length = int(sr * overlap_constant)
    duration = 600

    print(f"Audio length: {duration} seconds")

    num_blocks = int(duration / overlap_constant) - 1
    features = []

    for i in range(num_blocks): # for every 48000 samples
        start = i * hop_length
        end = start + block_length
        block = x[start:end]

        chroma = librosa.feature.chroma_stft(y=np.float32(block), sr=sr)
        chroma_mean = np.mean(chroma, axis=1)
        #print(chroma_mean)
        features.append(chroma_mean)

    return np.array(features)

In [ ]:
def get_features__(input_audio_path):
    y, sr = librosa.load(input_audio_path, sr=16000)
    print("sr: ", sr)
    block_length = sr # 48000 samples = 1 second
    overlap_constant = 0.6354
    hop_length = int(sr * overlap_constant)
    duration = librosa.get_duration(y=y, sr=sr)

    print(f"Audio length: {duration} seconds")

    num_blocks = int(duration / overlap_constant) - 1
    features = []

    for i in range(num_blocks): # for every 48000 samples
        start = i * hop_length
        end = start + block_length
        block = y[start:end]

        rms = librosa.feature.rms(y=block)
        features.append(rms)
        print(rms)

    return np.array(features)